In [1]:
import tensorflow as tf

# 1.simple placement

In [2]:
with tf.device("/cpu:0"):
    a = tf.Variable(3.0)
    b = tf.Variable(4.0)
    c = a + b

In [3]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    val = sess.run(c)

In [4]:
# Dynamic placement and soft placement

In [5]:
def variables_on_cpu(op):
    if op.type == "Variable":
        return "/gpu:0"
    else:
        return "/cpu:0"

with tf.device(variables_on_cpu):
    a = tf.Variable(3.0)
    b = tf.constant(4.0)
    c = a * b

config = tf.ConfigProto()
config.allow_soft_placement = True
with tf.Session(config=config) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    c_val = sess.run(c)

In [6]:
# Control Dependencies

In [7]:
with tf.variable_scope("control_dependencies"):
    a = tf.constant(1.0)
    b = a + 2.0

    with tf.control_dependencies([a, b]):
        x = tf.constant(3.0)
        y = tf.constant(4.0)

    z = x + y

In [8]:
# 2. Place on different servers

In [9]:
cluster_spec = tf.train.ClusterSpec({
    "ps": [
        "localhost:2221",  # /job:ps/task:0
    ],
    "worker": [
        "localhost:2222",  # /job:worker/task:0
        "localhost:2223",  # /job:worker/task:1
]})

In [10]:
#server = tf.train.Server(cluster_spec, job_name="worker", task_index=0)
#server.join()

In [11]:
# with tf.Session("grpc://machine-b.example.com:2222") as sess:
#     a = tf.constant(1.0)
#     b = tf.constant(2.0)
#     c = a + b
#     c.eval()

# 2. Sharing state

In [16]:
with tf.container("my_problem1"):
    a = tf.constant(1.0)

In [17]:
q = tf.FIFOQueue(capacity=10, dtypes=[tf.float32], shapes=[[2]], name="q", shared_name="shared_q")

# 3. Loading data

In [21]:
# 3.1
training_set_init = tf.placeholder(tf.float32, shape=(2, 3))
training_set = tf.Variable(training_set_init, trainable=False, collections=[], name="training_set")

In [22]:
training_set.shape

TensorShape([Dimension(2), Dimension(3)])

In [24]:
# 3.2 load with reader

In [25]:
# with reader
reader = tf.TextLineReader(skip_header_lines=1)
filename_queue = tf.FIFOQueue(capacity=10, dtypes=[tf.string], shapes=[()])
filename = tf.placeholder(tf.string)
enqueue_filename = filename_queue.enqueue([filename])
close_filename_queue = filename_queue.close()

In [26]:
key, value = reader.read(filename_queue)

In [27]:
x1, x2, target = tf.decode_csv(value, record_defaults=[[-1.], [-1.], [-1]])
features = tf.stack([x1, x2])

In [28]:
instance_queue = tf.RandomShuffleQueue(
    capacity=10, min_after_dequeue=2,
    dtypes=[tf.float32, tf.int32], shapes=[[2],[]],
    name="instance_q", shared_name="shared_instance_q")
enqueue_instance = instance_queue.enqueue([features, target])
close_instance_queue = instance_queue.close()

In [29]:
with tf.Session() as sess:
    sess.run(enqueue_filename, feed_dict={filename: "./my_test.csv"})
    sess.run(close_filename_queue)
    try:
        while True:
            sess.run(enqueue_instance)
    except tf.errors.OutOfRangeError as ex:
        pass # no more records in the current file and no more files to read
    sess.run(close_instance_queue)